In [25]:
import requests
from datetime import datetime
import pandas as pd
import json

response = requests.get("https://ogd-static.voteinfo-app.ch/v1/ogd/sd-t-17-02-20240922-eidgAbstimmung.json")
# change to new URL, make this code optional
# response = requests.get("https://ogd-static.voteinfo-app.ch/v1/ogd/sd-t-17-02-20241124-eidgAbstimmung.json")

# Check if request was successful
if response.status_code == 200:
    # Parse JSON data
    bfs_data = response.json()
    
    # Now you can work with data as a Python dictionary
    print("Data successfully loaded")
    with open('voting_data.json', 'w', encoding='utf-8') as f:
        json.dump(bfs_data, f, ensure_ascii=False, indent=2)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

# old_vorlagen = [{'id': 6710,
#   'title': 'Volksinitiative «Für die Zukunft unserer Natur und Landschaft (Biodiversitätsinitiative)»'},
#  {'id': 6720,
#   'title': 'Änderung des Bundesgesetzes über die berufliche Alters-, Hinterlassenen- und Invalidenvorsorge (BVG) (Reform der beruflichen Vorsorge)'}]

# current_vorlagen = [{'id': 6730,
#   'title': 'Bundesbeschluss über den Ausbauschritt 2023 für die Nationalstrassen'},
#  {'id': 6740,
#   'title': 'Änderung des Obligationenrechts (Mietrecht: Untermiete)'},
#  {'id': 6750,
#   'title': 'Änderung des Obligationenrechts (Mietrecht: Kündigung wegen Eigenbedarfs)'},
#  {'id': 6760,
#   'title': 'Änderung des Bundesgesetzes über die Krankenversicherung (KVG) (Einheitliche Finanzierung der Leistungen)'}]

def get_short_title(title):
   import re
   match = re.search(r'\((.*?)\)', title)
   return match.group(1) if match else title

vorlagen = [{
   'id': vorlage['vorlagenId'],
   'title_de': get_short_title(next(t['text'] for t in vorlage['vorlagenTitel'] if t['langKey'] == 'de')),
   'title_fr': get_short_title(next(t['text'] for t in vorlage['vorlagenTitel'] if t['langKey'] == 'fr'))
} for vorlage in bfs_data['schweiz']['vorlagen']]

results_df = pd.DataFrame(columns=['BfS_number', 'time', 'percent_yes', 'percent_no', 'lower_bound', 'upper_bound', 'share_counted', 'label', 'result', 'title_de', 'title_fr', 'type'])

for i, vorlage in enumerate(vorlagen):
   timestamp = int(datetime.now().timestamp() * 1000)
   url = f"https://tamedia-abstimmungszentrale-hochrechnung.storage.googleapis.com/{vorlage['id']}.csv?{timestamp}"
   leewas_data = pd.read_csv(url)
   
   latest_row = leewas_data.sort_values('Time').iloc[-1]
   
   percent_yes = latest_row['Estimate']
   result_type = 'trend'
   
   if bfs_data['schweiz']['vorlagen'][i]['resultat']['gebietAusgezaehlt']:
       percent_yes = bfs_data['schweiz']['vorlagen'][i]['resultat']['jaStimmenInProzent']
       result_type = 'final'
   
   new_row = pd.DataFrame({
       'BfS_number': [latest_row['BfS_number']],
       'time': [latest_row['Time']], 
       'percent_yes': [round(percent_yes, 2)],
       'percent_no': [round(100 - percent_yes, 2)],
       'lower_bound': [latest_row['lower_bound']],
       'upper_bound': [latest_row['upper_bound']],
       'share_counted': [latest_row['Share_counted']],
       'label': [latest_row['label']],
       'result': [latest_row['result']],
       'title_de': [vorlage['title_de']],
       'title_fr': [vorlage['title_fr']],
       'type': [result_type]
   })
   
   results_df = pd.concat([results_df, new_row], ignore_index=True)

results_df.to_csv("leewas_results.csv", index=False)

Data successfully loaded


/var/folders/c8/bvk5bjs15mzg5h8m_jcbywx40000gn/T/ipykernel_35144/1484331187.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
